<a href="https://colab.research.google.com/github/dk-wei/funcy/blob/main/Funcy_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install funcy

# Overview

Import stuff from funcy to make things happen:

`from funcy import whatever, you, need`

Merge collections of same type (works for dicts, sets, lists, tuples, iterators and even strings):
```
merge(coll1, coll2, coll3, ...)
join(colls)
merge_with(sum, dict1, dict2, ...)
```
Walk through collection, creating its transform (like map but preserves type):
```
walk(str.upper, {'a', 'b'})            # {'A', 'B'}
walk(reversed, {'a': 1, 'b': 2})       # {1: 'a', 2: 'b'}
walk_keys(double, {'a': 1, 'b': 2})    # {'aa': 1, 'bb': 2}
walk_values(inc, {'a': 1, 'b': 2})     # {'a': 2, 'b': 3}
```
Select a part of collection:
```
select(even, {1,2,3,10,20})                  # {2,10,20}
select(r'^a', ('a','b','ab','ba'))           # ('a','ab')
select_keys(callable, {str: '', None: None}) # {str: ''}
compact({2, None, 1, 0})                     # {1,2}
```
Manipulate sequences:
```
take(4, iterate(double, 1)) # [1, 2, 4, 8]
first(drop(3, count(10)))   # 13

lremove(even, [1, 2, 3])    # [1, 3]
lconcat([1, 2], [5, 6])     # [1, 2, 5, 6]
lcat(map(range, range(4)))  # [0, 0, 1, 0, 1, 2]
lmapcat(range, range(4))    # same
flatten(nested_structure)   # flat iter
distinct('abacbdd')         # iter('abcd')

lsplit(odd, range(5))       # ([1, 3], [0, 2, 4])
lsplit_at(2, range(5))      # ([0, 1], [2, 3, 4])
group_by(mod3, range(5))    # {0: [0, 3], 1: [1, 4], 2: [2]}

lpartition(2, range(5))     # [[0, 1], [2, 3]]
chunks(2, range(5))         # iter: [0, 1], [2, 3], [4]
pairwise(range(5))          # iter: [0, 1], [1, 2], ...
```
And functions:
```
partial(add, 1)             # inc
curry(add)(1)(2)            # 3
compose(inc, double)(10)    # 21
complement(even)            # odd
all_fn(isa(int), even)      # is_even_int

one_third = rpartial(operator.div, 3.0)
has_suffix = rcurry(str.endswith)
```
Create decorators easily:
```
@decorator
def log(call):
    print call._func.__name__, call._args
    return call()
```
Abstract control flow:
```
walk_values(silent(int), {'a': '1', 'b': 'no'})
# => {'a': 1, 'b': None}

@once
def initialize():
    "..."

with suppress(OSError):
    os.remove('some.file')

@ignore(ErrorRateExceeded)
@limit_error_rate(fails=5, timeout=60)
@retry(tries=2, errors=(HttpError, ServiceDown))
def some_unreliable_action(...):
    "..."

class MyUser(AbstractBaseUser):
    @cached_property
    def public_phones(self):
        return self.phones.filter(public=True)
```
Ease debugging:
```
squares = {tap(x, 'x'): tap(x * x, 'x^2') for x in [3, 4]}
# x: 3
# x^2: 9
# ...

@print_exits
def some_func(...):
    "..."

@log_calls(log.info, errors=False)
@log_errors(log.exception)
def some_suspicious_function(...):
    "..."

with print_durations('Creating models'):
    Model.objects.create(...)
    # ...
# 10.2 ms in Creating models
```

In [3]:
import funcy as fc

# flatten

一切皆可flatten，🐂🍺

In [7]:
l1 = [[1,2,3], [3,4,5],[3, [2,3, [3,2,4,5],4],4,5]]
list(fc.flatten(l1))

[1, 2, 3, 3, 4, 5, 3, 2, 3, 3, 2, 4, 5, 4, 4, 5]

# interpose
插入到两两元素之间

In [9]:
list(fc.interpose(None, [1,2,3])

[1, None, 2, None, 3]

# without

简单粗暴，直接删除指定items

In [15]:
lst = [1,3,4,5]

list(fc.without(lst,1,2,8))

[3, 4, 5]

# remove

按条件删除items

In [28]:
list(fc.remove(lambda x: x == None or x == 4 or x%2 ==0, [None, 1,2,3,4]))

[1, 3]

# group_by

按照标准划分group，例如下例标准为length

In [40]:
stats = fc.group_by(len, ['a', 'bb', 'c', 'rrr', 'e'])

print(stats[1])
print(stats[2])
print(stats[3])

['a', 'c', 'e']
['bb']
['rrr']


# partition & chunks # pairwise

按照要求进行chunk

In [42]:
list(fc.partition(4, list(range(21))))

[[0, 1, 2, 3],
 [4, 5, 6, 7],
 [8, 9, 10, 11],
 [12, 13, 14, 15],
 [16, 17, 18, 19]]

In [43]:
list(fc.chunks(4, list(range(21))))

[[0, 1, 2, 3],
 [4, 5, 6, 7],
 [8, 9, 10, 11],
 [12, 13, 14, 15],
 [16, 17, 18, 19],
 [20]]

In [46]:
list(fc.pairwise(list(range(11))))

[(0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10)]

# merge

合并多个列表

In [57]:
print('列表', list(fc.merge([1,2], [2,3])))

print('集合', list(fc.merge({1,2},{3,4,2},{1,2,6,7},{2,3,5,9})))

print('字典', fc.merge(*[{1:1,2:2},{1:23,4:3}]))

列表 [1, 2, 2, 3]
集合 [1, 2, 3, 4, 5, 6, 7, 9]
字典 {1: 23, 2: 2, 4: 3}


# merge_with

values合并，可以sum也可以别的

In [95]:
fc.merge_with(sum, {1:3, 3:10}, {3:5})

{1: 3, 3: 15}

# walk

类似map，但是保持type

In [104]:
print(fc.walk(str.upper, {'a', 'b'}))            # {'A', 'B'}
print(fc.walk(reversed, {'a': 1, 'b': 2}))       # {1: 'a', 2: 'b'}
#fc.walk_keys(double, {'a': 1, 'b': 2})    # {'aa': 1, 'bb': 2}
#fc.walk_values(inc, {'a': 1, 'b': 2})     # {'a': 2, 'b': 3}

{'A', 'B'}
{1: 'a', 2: 'b'}


# select

类似filter

In [103]:
#select(even, {1,2,3,10,20})                  # {2,10,20}
print(fc.select(r'^a', ('a','b','ab','ba')))           # ('a','ab')
#select_keys(callable, {str: '', None: None}) # {str: ''}
print(fc.compact({2, None, 1, 0}))                     # {1,2}

('a', 'ab')
{1, 2}


# silent

Ignore all real exceptions

In [98]:
# 对 `1`使用int function，如果正常就输出1，否则输出None
fc.silent(int)('1')   # 正常

1

In [62]:
fc.silent(int)('abc') == None   # 错误，但是不报错

True

# tap

list comprehension也能照样print结果

In [70]:
[fc.tap(i**2, 'x^2') for i in range(5)]

x^2: 0
x^2: 1
x^2: 4
x^2: 9
x^2: 16


[0, 1, 4, 9, 16]

# once

让某个方程在全局只执行一次

In [71]:
@fc.once
def func(x):
  return x**2

print(func(5))

print(func(5))

25
None


# memoize

记忆方程输出，即使中途被打断，依然记得之前所有的值

In [90]:
%%time

import math

@fc.memoize
def func(x):
  return 12/x

[fc.tap(func(x)) for x in [1,4,6,0,5,4]]

12.0
3.0
2.0


ZeroDivisionError: ignored

In [91]:
func.memory

{(1,): 12.0, (4,): 3.0, (6,): 2.0}

In [92]:
# 填充到memory
func.memory.update({(2, ): func(2)})
func.memory

{(1,): 12.0, (2,): 6.0, (4,): 3.0, (6,): 2.0}

In [93]:
func.memory.clear()
func.memory

{}